In [1]:
from bs4 import BeautifulSoup as soup  # HTML data structure
from urllib.request import urlretrieve, urlopen as uReq  # Web client
from time import time
import os
from selenium import webdriver
from multiprocessing import Process, Value, Pool
import datetime
import shutil
from selenium.webdriver.chrome.options import Options

In [2]:
def saveImages(car_id, car_soup):
    # create specific folder
    if os.path.isdir(car_id):
        shutil.rmtree(car_id)
    os.mkdir(car_id)
    os.chdir(car_id)

    # image processing...
    images = car_soup.findAll('div', {'class': 'thumbnail-image'})
    for ind, image in enumerate(images):
        img_src = image.find('img')['src']
        urlretrieve(img_src, "{}.jpg".format(ind))

    # end processing
    os.chdir('..')

In [3]:
def saveCSV(car_url, car_data_dict, car_soup):
    # FEATURE: car ID
    car_id = car_url.split('/')[5]
    car_data_dict['Id'] = car_id

    # FEATURE: car Price
    gel_price, usd_price = car_soup.findAll('span', {'class': 'car-price'})
    usd_price = usd_price.text
    if usd_price != 'Price negotiable':
        usd_price = usd_price.split()[0]
        usd_price = usd_price.replace(',', '')
        car_data_dict['Price'] = usd_price
    else:
        car_data_dict['Price'] = 'Price negotiable'

    # FEATURE: car customs
    custom_soup = car_soup.find('div', {'class': 'levy'})
    if custom_soup:
        custom = custom_soup.findAll('span')[1].findAll('span')[1].text
        custom = custom.split()[0]
        custom = custom.replace(',', '')
        car_data_dict['Customs'] = custom
    else:
        car_data_dict['Customs'] = ''

    # FEATURE: Data from <tr>-s
    car_data_table = car_soup.findAll('tr')
    for data in car_data_table:
        th1, th2 = data.findAll('th')

        # th1
        th1_divs = th1.findAll('div')
        if len(th1_divs) == 2:
            th1_key, th1_value = map(
                lambda x: x.text.strip(), th1_divs)
            if th1_key in car_data_dict:
                car_data_dict[th1_key] = th1_value

        # th2
        th2_divs = th2.findAll('div')
        if len(th2_divs) == 2:
            th2_key, th2_value = th2_divs
            th2_key = th2_key.text.strip()
            if 'class' in th2_value.i.attrs and 'fa-check' in th2_value.i['class']:
                res = '1'
            else:
                res = '0'

            th2_value = res
            if th2_key in car_data_dict:
                car_data_dict[th2_key] = th2_value

In [4]:
def scrapLink(data,):
    data_headers = ['Id', 'Manufacturer', 'Model', 'Category', 'Mileage', 'Gear box type', 'Doors',
                    'Wheel', 'Color', 'Interior color', 'VIN', 'Leather interior', 'Price', 'Customs']

    start = time()
    PID, links = data
    
    for ind, car_url in enumerate(links):
        now = time()
        ind += 1
        elapsed = int(now-start)
        if elapsed == 0:
            elapsed =1 
        mean = (elapsed/ind)
        estimated = (len(links)-ind)//mean
        print('Thread {}: Cars {}/{} -- Time Elapsed {}s -- Time Left {}s'.format(PID, ind, len(links), elapsed, estimated))
            
        try:
            # go to each car's url
            uClient = uReq(car_url)
            car_soup = soup(uClient.read(), "html.parser")
            uClient.close()


            if car_soup.find('div', {'class': 'error-wrapper'}):
                continue
            # DATA
            car_data_dict = {header: '' for header in data_headers}

            saveCSV(car_url, car_data_dict, car_soup)
            saveImages(car_data_dict['Id'], car_soup)
            # create string
            cur_data_str = ','.join([car_data_dict[x].replace('\n', ' ')
                                     for x in data_headers]) + '\n'
        except:
            print("Error while processing..")
    return

In [5]:
# read links
linksFile = open('SabaLinks.txt', 'r')
links = [line.rstrip() for line in linksFile]
linksFile.close()

folder = 'dataTest'
if os.path.isdir(folder):
    shutil.rmtree(folder)
os.mkdir(folder)
os.chdir(folder)

data_headers = ['Id', 'Manufacturer', 'Model', 'Category', 'Mileage', 'Gear box type', 'Doors',
                'Wheel', 'Color', 'Interior color', 'VIN', 'Leather interior', 'Price', 'Customs']

In [6]:
# write headers
f = open('cars.csv', "w", encoding='utf-8')
data_headers_str = ','.join(data_headers) + '\n'
f.write(data_headers_str)

121

In [7]:
procces_count = 20
links_for_threads = []
chunks_size = len(links) // procces_count
    
# divide data into threads
for i in range(procces_count):
    if i == procces_count - 1:
        links_for_threads += [[i, links[chunks_size * i:]]]
    else:
        links_for_threads += [[i, links[chunks_size * i: chunks_size * (i + 1)]]]



In [8]:
p = Pool(procces_count)
# initialize and run threads
threads = []
counter = Value('i', 0)
records = p.map(scrapLink, links_for_threads)
p.terminate()
p.join()

f.close()
exit()

Thread 0: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 1: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 2: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 4: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 5: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 13: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 12: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 6: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 15: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 3: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 8: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 18: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 17: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 14: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 9: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
Thread 16: Cars 1/1423 -- Time Elapsed 1s -- Time Left 1422.0s
T

Thread 4: Cars 7/1423 -- Time Elapsed 12s -- Time Left 826.0s
Thread 13: Cars 7/1423 -- Time Elapsed 12s -- Time Left 826.0s
Thread 12: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 2: Cars 8/1423 -- Time Elapsed 13s -- Time Left 870.0s
Thread 7: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 8: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 6: Cars 8/1423 -- Time Elapsed 13s -- Time Left 870.0s
Thread 15: Cars 8/1423 -- Time Elapsed 13s -- Time Left 870.0s
Thread 17: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 11: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 9: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 5: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 19: Cars 7/1427 -- Time Elapsed 13s -- Time Left 764.0s
Thread 10: Cars 7/1423 -- Time Elapsed 13s -- Time Left 762.0s
Thread 3: Cars 8/1423 -- Time Elapsed 13s -- Time Left 870.0s
Thread 18: Cars 10/1423 -- Time Elapsed 14s -- Time Left 1009.0

Thread 11: Cars 13/1423 -- Time Elapsed 27s -- Time Left 678.0s
Thread 5: Cars 13/1423 -- Time Elapsed 27s -- Time Left 678.0s
Thread 6: Cars 15/1423 -- Time Elapsed 27s -- Time Left 782.0s
Thread 7: Cars 14/1423 -- Time Elapsed 27s -- Time Left 730.0s
Thread 3: Cars 14/1423 -- Time Elapsed 27s -- Time Left 730.0s
Thread 18: Cars 16/1423 -- Time Elapsed 27s -- Time Left 833.0s
Thread 17: Cars 13/1423 -- Time Elapsed 28s -- Time Left 654.0s
Thread 15: Cars 16/1423 -- Time Elapsed 28s -- Time Left 804.0s
Thread 4: Cars 14/1423 -- Time Elapsed 28s -- Time Left 704.0s
Thread 9: Cars 13/1423 -- Time Elapsed 28s -- Time Left 654.0s
Thread 8: Cars 14/1423 -- Time Elapsed 28s -- Time Left 704.0s
Thread 13: Cars 13/1423 -- Time Elapsed 28s -- Time Left 654.0s
Thread 10: Cars 13/1423 -- Time Elapsed 28s -- Time Left 654.0s
Thread 1: Cars 14/1423 -- Time Elapsed 28s -- Time Left 704.0s
Thread 19: Cars 13/1427 -- Time Elapsed 28s -- Time Left 656.0s
Thread 2: Cars 15/1423 -- Time Elapsed 28s -- Ti

Thread 0: Cars 21/1423 -- Time Elapsed 40s -- Time Left 736.0s
Thread 15: Cars 24/1423 -- Time Elapsed 40s -- Time Left 839.0s
Thread 8: Cars 20/1423 -- Time Elapsed 40s -- Time Left 701.0s
Thread 14: Cars 19/1423 -- Time Elapsed 40s -- Time Left 666.0s
Thread 12: Cars 20/1423 -- Time Elapsed 40s -- Time Left 701.0s
Thread 10: Cars 19/1423 -- Time Elapsed 40s -- Time Left 666.0s
Thread 9: Cars 19/1423 -- Time Elapsed 40s -- Time Left 666.0s
Thread 1: Cars 20/1423 -- Time Elapsed 40s -- Time Left 701.0s
Thread 6: Cars 22/1423 -- Time Elapsed 40s -- Time Left 770.0s
Thread 17: Cars 19/1423 -- Time Elapsed 40s -- Time Left 666.0s
Thread 18: Cars 23/1423 -- Time Elapsed 41s -- Time Left 785.0s
Thread 3: Cars 21/1423 -- Time Elapsed 41s -- Time Left 718.0s
Thread 7: Cars 21/1423 -- Time Elapsed 41s -- Time Left 718.0s
Thread 4: Cars 21/1423 -- Time Elapsed 41s -- Time Left 718.0s
Thread 16: Cars 23/1423 -- Time Elapsed 41s -- Time Left 785.0s
Thread 11: Cars 21/1423 -- Time Elapsed 41s -- T

Thread 1: Cars 26/1423 -- Time Elapsed 52s -- Time Left 698.0s
Thread 13: Cars 26/1423 -- Time Elapsed 52s -- Time Left 698.0s
Thread 19: Cars 25/1427 -- Time Elapsed 53s -- Time Left 661.0s
Thread 2: Cars 27/1423 -- Time Elapsed 53s -- Time Left 711.0s
Thread 6: Cars 29/1423 -- Time Elapsed 53s -- Time Left 762.0s
Thread 8: Cars 27/1423 -- Time Elapsed 53s -- Time Left 711.0s
Error while processing..
Thread 2: Cars 28/1423 -- Time Elapsed 53s -- Time Left 736.0s
Thread 14: Cars 26/1423 -- Time Elapsed 53s -- Time Left 685.0s
Thread 12: Cars 27/1423 -- Time Elapsed 53s -- Time Left 711.0s
Thread 16: Cars 30/1423 -- Time Elapsed 53s -- Time Left 788.0s
Thread 0: Cars 27/1423 -- Time Elapsed 53s -- Time Left 711.0s
Thread 10: Cars 26/1423 -- Time Elapsed 53s -- Time Left 685.0s
Thread 11: Cars 28/1423 -- Time Elapsed 54s -- Time Left 723.0s
Thread 9: Cars 26/1423 -- Time Elapsed 54s -- Time Left 672.0s
Thread 4: Cars 29/1423 -- Time Elapsed 54s -- Time Left 748.0s
Thread 17: Cars 26/1423

...

Thread 17: Cars 1344/1423 -- Time Elapsed 2440s -- Time Left 43.0s
Error while processing..
Thread 9: Cars 1385/1423 -- Time Elapsed 2440s -- Time Left 21.0s
Error while processing..
Thread 16: Cars 1388/1423 -- Time Elapsed 2440s -- Time Left 19.0s
Error while processing..
Thread 14: Cars 1420/1423 -- Time Elapsed 2440s -- Time Left 1.0s
Error while processing..
Thread 11: Cars 1376/1423 -- Time Elapsed 2440s -- Time Left 26.0s
Error while processing..
Thread 12: Cars 1417/1423 -- Time Elapsed 2440s -- Time Left 3.0s
Error while processing..
Thread 2: Cars 1390/1423 -- Time Elapsed 2440s -- Time Left 18.0s
Error while processing..
Thread 9: Cars 1386/1423 -- Time Elapsed 2441s -- Time Left 21.0s
Error while processing..
Thread 17: Cars 1345/1423 -- Time Elapsed 2441s -- Time Left 42.0s
Error while processing..
Thread 16: Cars 1389/1423 -- Time Elapsed 2441s -- Time Left 19.0s
Error while processing..
Thread 14: Cars 1421/1423 -- Time Elapsed 2441s -- Time Left 1.0s
Error while process

Thread 11: Cars 1391/1423 -- Time Elapsed 2447s -- Time Left 18.0s
Error while processing..
Thread 17: Cars 1360/1423 -- Time Elapsed 2447s -- Time Left 35.0s
Error while processing..
Thread 9: Cars 1401/1423 -- Time Elapsed 2447s -- Time Left 12.0s
Error while processing..
Thread 2: Cars 1406/1423 -- Time Elapsed 2447s -- Time Left 9.0s
Error while processing..
Thread 16: Cars 1403/1423 -- Time Elapsed 2447s -- Time Left 11.0s
Error while processing..
Thread 17: Cars 1361/1423 -- Time Elapsed 2447s -- Time Left 34.0s
Error while processing..
Thread 11: Cars 1392/1423 -- Time Elapsed 2447s -- Time Left 17.0s
Error while processing..
Thread 9: Cars 1402/1423 -- Time Elapsed 2447s -- Time Left 12.0s
Error while processing..
Thread 16: Cars 1404/1423 -- Time Elapsed 2447s -- Time Left 10.0s
Error while processing..
Thread 2: Cars 1407/1423 -- Time Elapsed 2447s -- Time Left 9.0s
Error while processing..
Thread 17: Cars 1362/1423 -- Time Elapsed 2448s -- Time Left 33.0s
Error while process

Thread 16: Cars 1421/1423 -- Time Elapsed 2457s -- Time Left 1.0s
Error while processing..
Thread 17: Cars 1379/1423 -- Time Elapsed 2457s -- Time Left 24.0s
Error while processing..
Thread 2: Cars 1423/1423 -- Time Elapsed 2457s -- Time Left 0.0s
Error while processing..
Thread 9: Cars 1419/1423 -- Time Elapsed 2457s -- Time Left 2.0s
Error while processing..
Thread 11: Cars 1409/1423 -- Time Elapsed 2457s -- Time Left 8.0s
Error while processing..
Error while processing..
Thread 9: Cars 1420/1423 -- Time Elapsed 2457s -- Time Left 1.0s
Thread 16: Cars 1422/1423 -- Time Elapsed 2457s -- Time Left 0.0s
Error while processing..
Error while processing..
Error while processing..
Thread 11: Cars 1410/1423 -- Time Elapsed 2457s -- Time Left 7.0s
Thread 17: Cars 1380/1423 -- Time Elapsed 2457s -- Time Left 24.0s
Error while processing..
Error while processing..
Thread 16: Cars 1423/1423 -- Time Elapsed 2458s -- Time Left 0.0s
Thread 9: Cars 1421/1423 -- Time Elapsed 2458s -- Time Left 1.0s
E

Error while processing..
Thread 8: Cars 1383/1423 -- Time Elapsed 2493s -- Time Left 22.0s
Error while processing..
Thread 8: Cars 1384/1423 -- Time Elapsed 2493s -- Time Left 21.0s
Error while processing..
Thread 8: Cars 1385/1423 -- Time Elapsed 2494s -- Time Left 21.0s
Error while processing..
Thread 8: Cars 1386/1423 -- Time Elapsed 2494s -- Time Left 20.0s
Error while processing..
Thread 8: Cars 1387/1423 -- Time Elapsed 2495s -- Time Left 20.0s
Error while processing..
Thread 8: Cars 1388/1423 -- Time Elapsed 2495s -- Time Left 19.0s
Error while processing..
Thread 8: Cars 1389/1423 -- Time Elapsed 2496s -- Time Left 18.0s
Error while processing..
Thread 8: Cars 1390/1423 -- Time Elapsed 2496s -- Time Left 18.0s
Error while processing..
Thread 8: Cars 1391/1423 -- Time Elapsed 2496s -- Time Left 17.0s
Error while processing..
Thread 8: Cars 1392/1423 -- Time Elapsed 2497s -- Time Left 17.0s
Error while processing..
Thread 8: Cars 1393/1423 -- Time Elapsed 2497s -- Time Left 16.0s

NameError: name 'driver' is not defined